In [1]:
import pandas as pd
import csv
import numpy as np

#Load Data
dfDBPedia = pd.read_csv('DBPediaCity.csv', sep=',')
dfGeonames = pd.read_csv('GeonamesData.csv', sep=',')
dfWebTables = pd.read_csv('WebTableDataCity.csv', sep=',')
dfWiki = pd.read_csv('WikiDataCity.csv', sep=',')

#Lowercase identifier
dfDBPedia['URI'] = dfDBPedia['URI'].str.lower()
dfGeonames['URI'] = dfGeonames['URI'].str.lower()
dfWebTables['URI'] = dfWebTables['URI'].str.lower()
dfWiki['uri'] = dfWiki['uri'].str.lower()

#Load True Matches
dfGeonamesGoldStandard = pd.read_csv('goldstandard_city_true_GeonamesData.csv', sep=',', header=None)
dfWebTablesGoldStandard = pd.read_csv('goldstandard_city_true_WebTableDataCity.csv', sep=',', header=None)
dfWikiGoldStandard = pd.read_csv('goldstandard_city_true_WikiDataCity.csv', sep=',', header=None)

In [ ]:
display(dfDBPedia.describe(include='all'))

In [ ]:
dfDBPedia['latitude'] = dfDBPedia['latitude'].astype('str')
dfDBPedia['longitude'] = dfDBPedia['longitude'].astype('str')
dfDBPedia['population'] = dfDBPedia['population'].astype('str')

dfDBPediaInfo = dfDBPedia.filter(items = ['country', 'label', 'latitude', 'longitude', 'population']).describe()

datatypes = {'country': ['string'], 'label': ['string'], 'latitude': ['coordinate'], 
             'longitude': ['coordinate'], 'population': ['numeric']}

dfDBPediaProfiling = pd.DataFrame(data= datatypes)
dfDBPediaProfiling = dfDBPediaProfiling.append((dfDBPediaInfo.loc[['count']] / len(dfDBPedia)).astype(np.float).round(3))
dfDBPediaProfiling = dfDBPediaProfiling.append((dfDBPediaInfo.loc[['unique']] / dfDBPediaInfo.loc[['count']].values).astype(np.float).round(3))

dfDBPediaProfiling.index = ['Data Type', 'Density', 'Uniqueness']

dfDBPediaProfiling.to_csv('DBPedia_city_profiling.csv', sep=',', encoding='utf-8', index=True, quotechar='"', quoting=csv.QUOTE_ALL)
print(dfDBPediaProfiling.transpose().filter(items=['Density', 'Uniqueness']).to_latex())

In [ ]:
dfGeonames['latitude'] = dfGeonames['latitude'].astype('str')
dfGeonames['longitude'] = dfGeonames['longitude'].astype('str')
dfGeonames['population'] = dfGeonames['population'].astype('str')

dfGeonamesInfo = dfGeonames.filter(items = ['country', 'label', 'latitude', 'longitude', 'population']).describe(include='all')

#display(dfDnbInfo)
datatypes = {'country': ['string'], 'label': ['string'], 'latitude': ['coordinate'], 
             'longitude': ['coordinate'], 'population': ['numeric']}

dfGeonamesProfiling = pd.DataFrame(data= datatypes)
dfGeonamesProfiling = dfGeonamesProfiling.append((dfGeonamesInfo.loc[['count']] / len(dfGeonames)).astype(np.float).round(3), sort = False)
dfGeonamesProfiling = dfGeonamesProfiling.append((dfGeonamesInfo.loc[['unique']] / dfGeonamesInfo.loc[['count']].values).astype(np.float).round(3), sort = False)

dfGeonamesProfiling.index = ['Data Type', 'Density', 'Uniqueness']

In [ ]:
from nltk import word_tokenize, ngrams
from nltk.metrics import jaccard_distance
import numpy as np

mapping = {"country": "country", "label": "label", "latitude": "latitude", "longitude": "longitude", "population": "population"}
jaccard_similarity_counter = {"country" : 0, "label": 0, "latitude":0, "longitude": 0, "population": 0}

for index, row in dfGeonamesGoldStandard.iterrows():
    rowDBPedia = dfDBPedia[dfDBPedia['URI'] == row[0]]
    rowGeonames = dfGeonames[dfGeonames['URI'] == row[1]]
    for key in mapping:
        if type(rowGeonames[key].values[0]) not in[np.float64, float] and type(rowDBPedia[mapping[key]].values[0]) not in[np.float64, float]:
            jaccard_similarity_counter[key] += jaccard_distance(set(ngrams(rowDBPedia[mapping[key]].values[0], 2)), set(ngrams(rowGeonames[key].values[0], 2)))
    
for key in jaccard_similarity_counter:
    jaccard_similarity_counter[key] = round(jaccard_similarity_counter[key] / len(dfGeonamesGoldStandard), 3)
 
#display(jaccard_similarity_counter)
dfGeonamesProfiling = dfGeonamesProfiling.append(pd.DataFrame(data=jaccard_similarity_counter, index=['Heterogeneity']))
dfGeonamesProfiling.to_csv('Dnb_city_profiling.csv', sep=',', encoding='utf-8', index=True, quotechar='"', quoting=csv.QUOTE_ALL)
print(dfGeonamesProfiling.transpose().filter(items=['Density', 'Uniqueness', 'Heterogeneity']).to_latex())

In [ ]:
display(dfWebTables)

In [2]:
dfWebTables['latitude'] = dfWebTables['latitude'].astype('str')
dfWebTables['longitude'] = dfWebTables['longitude'].astype('str')
dfWebTables['population'] = dfWebTables['population'].astype('str')

display(dfWebTables)

dfWebTablesInfo = dfWebTables.filter(items = ['country', 'label', 'latitude', 'longitude', 'population']).describe(include='all')
datatypes = {'country': ['string'], 'label': ['string'], 'latitude': ['coordinate'], 
             'longitude': ['coordinate'], 'population': ['numeric']}

dfWebTablesProfiling = pd.DataFrame(data= datatypes)
dfWebTablesProfiling = dfWebTablesProfiling.append((dfWebTablesInfo.loc[['count']] / len(dfWebTables)).astype(np.float).round(3), sort = False)
dfWebTablesProfiling = dfWebTablesProfiling.append((dfWebTablesInfo.loc[['unique']] / dfWebTablesInfo.loc[['count']].values).astype(np.float).round(3), sort = False)

dfWebTablesProfiling.index = ['Data Type', 'Density', 'Uniqueness']
display(dfWebTablesProfiling)

,URI,label,population,country,latitude,longitude
0,webtables_0,MacAllen,"110,000",USA,nan,nan
1,webtables_1,Maceio,"922,000",Brazil,nan,nan
2,webtables_2,Macheng,"1,880,000",China,nan,nan
3,webtables_3,MADRID,"3,213,000",Spain,nan,nan
4,webtables_4,Madurai,"1,130,000",India,nan,nan
5,webtables_5,Magdeburg,"230,000",Germany,nan,nan
6,webtables_6,Maiduguri,"1,200,000",Nigeria,nan,nan
7,webtables_7,Makasar,"1,113,000",Indonesia,nan,nan
8,webtables_8,Malang,"742,000",Indonesia,nan,nan
9,webtables_9,Malmo,"244,000",Sweden,nan,nan


,country,label,latitude,longitude,population
Data Type,string,string,coordinate,coordinate,numeric
Density,1,1,1,1,1
Uniqueness,0.135,0.72,0.001,0.001,0.595


In [3]:
from nltk import word_tokenize, ngrams
from nltk.metrics import jaccard_distance
import numpy as np

mapping = {"country": "country", "label": "label", "latitude": "latitude", "longitude": "longitude", "population": "population"}
jaccard_similarity_counter = {"country" : 0, "label": 0, "latitude":0, "longitude": 0, "population": 0}

for index, row in dfWebTablesGoldStandard.iterrows():
    rowDBPedia = dfDBPedia[dfDBPedia['URI'] == row[0]]
    rowWebTables = dfWebTables[dfWebTables['URI'] == row[1]]
    for key in mapping:
        if type(rowWebTables[key].values[0]) not in[np.float64, float] and type(rowDBPedia[mapping[key]].values[0]) not in[np.float64, float]:
            jaccard_similarity_counter[key] += jaccard_distance(set(ngrams(rowDBPedia[mapping[key]].values[0], 2)), set(ngrams(rowWebTables[key].values[0], 2)))
    
for key in jaccard_similarity_counter:
    jaccard_similarity_counter[key] = round(jaccard_similarity_counter[key] / len(dfWebTablesGoldStandard),3)
 
#display(jaccard_similarity_counter)
dfWebTablesProfiling = dfWebTablesProfiling.append(pd.DataFrame(data=jaccard_similarity_counter, index=['Heterogeneity']))
dfWebTablesProfiling.to_csv('webtable_city_profiling.csv', sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL)
display(dfWebTablesProfiling)
#cols = dfWebTablesProfiling.columns.tolist()
#cols = cols[1:] + cols[:1]
#dfWebTablesProfilingNew = dfWebTablesProfiling[cols]

print(dfWebTablesProfiling.transpose().filter(items=['Density', 'Uniqueness', 'Heterogeneity']).to_latex())

,country,label,latitude,longitude,population
Data Type,string,string,coordinate,coordinate,numeric
Density,1,1,1,1,1
Uniqueness,0.135,0.72,0.001,0.001,0.595
Heterogeneity,0.114,0.229,0,0,0


\begin{tabular}{llll}
\toprule
{} & Density & Uniqueness & Heterogeneity \\
\midrule
country    &       1 &      0.135 &         0.114 \\
label      &       1 &       0.72 &         0.229 \\
latitude   &       1 &      0.001 &             0 \\
longitude  &       1 &      0.001 &             0 \\
population &       1 &      0.595 &             0 \\
\bottomrule
\end{tabular}



In [ ]:
display(dfWiki)

In [ ]:
dfWiki['latitude'] = dfWiki['latitude'].astype('str')
dfWiki['longitude'] = dfWiki['longitude'].astype('str')
dfWiki['population'] = dfWiki['population'].astype('str')

dfWikiInfo = dfWiki.filter(items = ['country', 'label', 'latitude', 'longitude', 'population']).describe(include='all')
datatypes = {'country': ['string'], 'label': ['string'], 'latitude': ['coordinate'], 
             'longitude': ['coordinate'], 'population': ['numeric']}

dfWikiProfiling = pd.DataFrame(data= datatypes)
dfWikiProfiling = dfWikiProfiling.append((dfWikiInfo.loc[['count']] / len(dfWiki)).astype(np.float).round(3), sort = False)
dfWikiProfiling = dfWikiProfiling.append((dfWikiInfo.loc[['unique']] / dfWikiInfo.loc[['count']].values).astype(np.float).round(3), sort = False)

dfWikiProfiling.index = ['Data Type', 'Density', 'Uniqueness']
display(dfWikiProfiling)

In [ ]:
from nltk import word_tokenize, ngrams
from nltk.metrics import jaccard_distance
import numpy as np

mapping = {"country": "country", "label": "label", "latitude": "latitude", "longitude": "longitude", "population": "population"}
jaccard_similarity_counter = {"country" : 0, "label": 0, "latitude":0, "longitude": 0, "population": 0}

for index, row in dfWikiGoldStandard.iterrows():
    rowDBPedia = dfDBPedia[dfDBPedia['URI'] == row[0]]
    rowWiki = dfWiki[dfWiki['uri'] == row[1]]
    for key in mapping:
        if type(rowWiki[key].values[0]) not in[np.float64, float] and type(rowDBPedia[mapping[key]].values[0]) not in[np.float64, float]:
            jaccard_similarity_counter[key] += jaccard_distance(set(ngrams(rowDBPedia[mapping[key]].values[0], 2)), set(ngrams(rowWiki[key].values[0], 2)))
    
for key in jaccard_similarity_counter:
    jaccard_similarity_counter[key] = round(jaccard_similarity_counter[key] / len(dfWikiGoldStandard),3)
 
#display(jaccard_similarity_counter)
dfWikiProfiling = dfWikiProfiling.append(pd.DataFrame(data=jaccard_similarity_counter, index=['Heterogeneity']))
dfWikiProfiling.to_csv('wiki_city_profiling.csv', sep=',', encoding='utf-8', index=True, quotechar='"', quoting=csv.QUOTE_ALL)

print(dfWikiProfiling.transpose().filter(items=['Density', 'Uniqueness', 'Heterogeneity']).to_latex())

In [ ]:
print(dfWikiProfiling.to_latex(bold_rows=True, multicolumn=True))

In [ ]:
dfAggregatedProfiling = dfDBPediaProfiling.transpose().append(dfGeonamesProfiling.transpose())
dfAggregatedProfiling = dfAggregatedProfiling.append(dfWebTablesProfiling.transpose())
dfAggregatedProfiling = dfAggregatedProfiling.append(dfWikiProfiling.transpose())
display(dfAggregatedProfiling)
dfSummedProfiling = pd.DataFrame()
for index in set(dfAggregatedProfiling.index):
    profil = {}
    dfSubset = dfAggregatedProfiling.loc[index]
    display(dfSubset)
    profil['Attribute'] = index
    profil['Density-Mean'] = dfSubset['Density'].mean()
    profil['Density-Std'] = dfSubset['Density'].std()
    profil['Heterogeneity-Mean'] = dfSubset['Heterogeneity'].mean()
    profil['Heterogeneity-Std'] = dfSubset['Heterogeneity'].std()
    profil['Uniqueness-Mean'] = dfSubset['Uniqueness'].mean()
    profil['Uniqueness-Std'] = dfSubset['Uniqueness'].std()
    dfProfil = pd.DataFrame(profil, index=[0])
    dfSummedProfiling = dfSummedProfiling.append(dfProfil)

display(dfSummedProfiling)
dfSummedProfiling.to_csv('city_profiling.csv', sep=',', encoding='utf-8', index=True, quotechar='"', quoting=csv.QUOTE_ALL)